# Setup

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
dc_data_directory = "/content/drive/MyDrive/DC project 2/Trip data w taxi codes"

# Load uber data from directory

Load the data into a dictionary of the form: {filename1: df1, filename2: df2, ...}

In [4]:
uber_data = {}
for file in os.listdir(dc_data_directory):
  f = os.path.join(dc_data_directory, file)
  uber_data[f] = pd.read_csv(f)

# Get 2014 and 2015 data in the same format

In [5]:
# Columns and order for final dataframes
col_order = ['datetime', 'lat', 'lon', 'base', 'locationID', 'borough', 'zone']

## Identify differences between 2014 and 2015 data

The janjune15 data is in a different format than the other csvs. 

In [6]:
# 2014 data
uber_data['/content/drive/MyDrive/DC project 2/Trip data w taxi codes/uber_apr14_wcodes.csv'].head()

,Date/Time,Lat,Lon,Base,borough,location_i,zone
0,4/1/2014 4:20:00,40.6950,-74.1783,B02512,EWR,1.0,Newark Airport
1,4/1/2014 4:27:00,40.6879,-74.1814,B02512,EWR,1.0,Newark Airport
2,4/1/2014 4:38:00,40.6878,-74.1816,B02512,EWR,1.0,Newark Airport
3,4/1/2014 5:34:00,40.6907,-74.1782,B02512,EWR,1.0,Newark Airport
4,4/1/2014 7:50:00,40.6950,-74.1774,B02512,EWR,1.0,Newark Airport


In [7]:
# 2015 data
uber_data['/content/drive/MyDrive/DC project 2/Trip data w taxi codes/uber_janjune15_wcodes_centroids.csv'].head()

,Unnamed: 0,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID,borough,zone,xCentroid,yCentroid
0,1,B02617,2015/05/17 09:47:00,B02617,141,Manhattan,Lenox Hill West,-73.9596,40.7669
1,2,B02617,2015/05/17 09:47:00,B02617,65,Brooklyn,Downtown Brooklyn/MetroTech,-73.9861,40.6953
2,3,B02617,2015/05/17 09:47:00,B02617,100,Manhattan,Garment District,-73.9888,40.7535
3,4,B02617,2015/05/17 09:47:00,B02774,80,Brooklyn,East Williamsburg,-73.9368,40.7154
4,5,B02617,2015/05/17 09:47:00,B02617,90,Manhattan,Flatiron,-73.9970,40.7423


## Transform the 2015 data into the proper schema

### Transformation

In [ ]:
janjune15 = uber_data['/content/drive/MyDrive/DC project 2/Trip data w taxi codes/uber_janjune15_wcodes_centroids.csv']

# Convert date to datetime format
janjune15['datetime'] = pd.to_datetime(janjune15['Pickup_date'])

# Drop extraneous columns
janjune15 = janjune15.drop(['Unnamed: 0', 'Dispatching_base_num', 'Pickup_date'], axis = 1)

# Rename columns to obtain the same format
janjune15 = janjune15.rename(columns = {"Affiliated_base_num": "base", "xCentroid": "lon", "yCentroid":"lat"})

# Reorder the columns
janjune15 = janjune15[col_order]

# Sort by datetime
janjune15 = janjune15.sort_values("datetime").reset_index(drop = True)

# Final format
janjune15.head()

,datetime,lat,lon,base,locationID,borough,zone
0,2015-01-01 00:00:05,40.7188,-73.9574,B02764,255,Brooklyn,Williamsburg (North Side)
1,2015-01-01 00:00:05,40.7209,-73.9969,B02598,144,Manhattan,Little Italy/NoLiTa
2,2015-01-01 00:00:08,40.7326,-73.9943,B02682,113,Manhattan,Greenwich Village North
3,2015-01-01 00:00:11,40.7403,-73.9905,B02617,234,Manhattan,Union Sq
4,2015-01-01 00:00:13,40.7147,-73.9830,B02764,232,Manhattan,Two Bridges/Seward Park


In [ ]:
janjune15.info()

### Save the 2015 uber data

In [ ]:
# janjune15.to_csv(os.path.join(dc_data_directory, "uber_2015_processed.csv"), index = False)

### Confirm the 2015 data saved correctly

In [ ]:
pd.read_csv(os.path.join(dc_data_directory,"uber_2015_processed.csv")).head()

,datetime,lat,lon,base,locationID,borough,zone
0,2015-01-01 00:00:05,40.7188,-73.9574,B02764,255,Brooklyn,Williamsburg (North Side)
1,2015-01-01 00:00:05,40.7209,-73.9969,B02598,144,Manhattan,Little Italy/NoLiTa
2,2015-01-01 00:00:08,40.7326,-73.9943,B02682,113,Manhattan,Greenwich Village North
3,2015-01-01 00:00:11,40.7403,-73.9905,B02617,234,Manhattan,Union Sq
4,2015-01-01 00:00:13,40.7147,-73.9830,B02764,232,Manhattan,Two Bridges/Seward Park


## Get the 2014 data in the proper format

### Read the 2014 data

In [10]:
uber_2014_dfs = [] # store dataframes for all months in 2014

for filename in os.listdir(dc_data_directory):
  if ("14" in filename and "uber_2014_processed" not in filename): # only 2014 files
    print(filename)
    f = os.path.join(dc_data_directory, filename)
    df = pd.read_csv(f) 
    uber_2014_dfs.append(df) # save df

# Combine all 2014 data into a single dataframe
uber_2014_all = pd.concat(uber_2014_dfs, axis=0, ignore_index=True) 

uber_apr14_wcodes.csv
uber_sep2014_wcodes.csv
uber_may2014_wcodes.csv
uber_jun2014_wcodes.csv
uber_jul2014_wcodes.csv
uber_aug2014_wcodes.csv


In [11]:
uber_2014_all.head()

,Date/Time,Lat,Lon,Base,borough,location_i,zone
0,4/1/2014 4:20:00,40.6950,-74.1783,B02512,EWR,1.0,Newark Airport
1,4/1/2014 4:27:00,40.6879,-74.1814,B02512,EWR,1.0,Newark Airport
2,4/1/2014 4:38:00,40.6878,-74.1816,B02512,EWR,1.0,Newark Airport
3,4/1/2014 5:34:00,40.6907,-74.1782,B02512,EWR,1.0,Newark Airport
4,4/1/2014 7:50:00,40.6950,-74.1774,B02512,EWR,1.0,Newark Airport


Confirm there are no issues with 4. and 4.9999 in the location_i values (as Richard observed in qgis)

In [15]:
# Check for issues with location_i values from qgis
np.unique(uber_2014_all["location_i"])

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,
        68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,
        79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,
        90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99., 100.,
       101., 102., 103., 106., 107., 108., 109., 111., 112., 113., 114.,
       115., 116., 117., 118., 119., 120., 121., 122., 123., 124., 125.,
       126., 127., 128., 129., 130., 131., 132., 133., 134., 135., 136.,
       137., 138., 139., 140., 141., 142., 143., 144., 145., 146., 147.,
       148., 149., 150., 151., 152., 153., 154., 15

In [21]:
uber_2014_all[uber_2014_all.location_i == float(4)]["borough"].unique()

array(['Manhattan'], dtype=object)

### Transform the 2014 data

In [ ]:
# Convert date to datetime format
uber_2014_all['datetime'] = pd.to_datetime(uber_2014_all['Date/Time'], format="%m/%d/%Y %H:%M:%S")

# Convert location_i format
uber_2014_all['locationID'] = uber_2014_all['location_i'].astype(int)

# Drop extraneous columns
uber_2014_all = uber_2014_all.drop(['location_i', 'Date/Time'], axis = 1)

# Rename columns to obtain the same format
uber_2014_all = uber_2014_all.rename(columns = {"Lat": "lat", "Lon": "lon", "Base":"base"})

# Reorder the columns
uber_2014_all = uber_2014_all[col_order]

# Sort by datetime
uber_2014_all = uber_2014_all.sort_values("datetime").reset_index(drop = True)

In [ ]:
# Uber 2014 all final format
uber_2014_all.head()

,datetime,lat,lon,base,locationID,borough,zone
0,2014-04-01 00:00:00,40.7188,-73.9863,B02598,232,Manhattan,Two Bridges/Seward Park
1,2014-04-01 00:00:00,40.7637,-73.9600,B02598,141,Manhattan,Lenox Hill West
2,2014-04-01 00:00:00,40.7215,-73.9952,B02682,144,Manhattan,Little Italy/NoLiTa
3,2014-04-01 00:01:00,40.7355,-73.9966,B02617,113,Manhattan,Greenwich Village North
4,2014-04-01 00:02:00,40.7184,-73.9601,B02682,255,Brooklyn,Williamsburg (North Side)


### Save processed 2014 data to csv

In [ ]:
# uber_2014_all.to_csv(os.path.join(dc_data_directory, "uber_2014_processed.csv"), index = False)

### Confirm the 2014 data saved correctly

In [ ]:
pd.read_csv(os.path.join(dc_data_directory,"uber_2014_processed.csv")).head()

,datetime,lat,lon,base,locationID,borough,zone
0,2014-04-01 00:00:00,40.7188,-73.9863,B02598,232,Manhattan,Two Bridges/Seward Park
1,2014-04-01 00:00:00,40.7637,-73.9600,B02598,141,Manhattan,Lenox Hill West
2,2014-04-01 00:00:00,40.7215,-73.9952,B02682,144,Manhattan,Little Italy/NoLiTa
3,2014-04-01 00:01:00,40.7355,-73.9966,B02617,113,Manhattan,Greenwich Village North
4,2014-04-01 00:02:00,40.7184,-73.9601,B02682,255,Brooklyn,Williamsburg (North Side)


# Combine 2014 and 2015 uber data into a single file

### Read 2014 and 2015 data

In [ ]:
uber_2014 = pd.read_csv(os.path.join(dc_data_directory,"uber_2014_processed.csv"))
uber_2015 = pd.read_csv(os.path.join(dc_data_directory,"uber_2015_processed.csv"))

# concat 2014 and 2015 uber data
uber_all = pd.concat([uber_2014, uber_2015], axis = 0, ignore_index = True)

### Confirm format of combined data

In [ ]:
uber_all.head()

,datetime,lat,lon,base,locationID,borough,zone
0,2014-04-01 00:00:00,40.7188,-73.9863,B02598,232,Manhattan,Two Bridges/Seward Park
1,2014-04-01 00:00:00,40.7637,-73.9600,B02598,141,Manhattan,Lenox Hill West
2,2014-04-01 00:00:00,40.7215,-73.9952,B02682,144,Manhattan,Little Italy/NoLiTa
3,2014-04-01 00:01:00,40.7355,-73.9966,B02617,113,Manhattan,Greenwich Village North
4,2014-04-01 00:02:00,40.7184,-73.9601,B02682,255,Brooklyn,Williamsburg (North Side)


In [ ]:
print(uber_all["datetime"].min(), "\n", uber_all["datetime"].max())

2014-04-01 00:00:00 
 2015-06-30 23:59:00


### Save all 2014 and 2015 data into a single file

In [ ]:
# uber_all.to_csv(os.path.join(dc_data_directory, "uber_all_processed.csv"), index = False)

### Confirm the uber data saved correctly

In [ ]:
uber_all = pd.read_csv(os.path.join(dc_data_directory,"uber_all_processed.csv"), parse_dates = ["datetime"])

In [ ]:
uber_all.head()

,datetime,lat,lon,base,locationID,borough,zone
0,2014-04-01 00:00:00,40.7188,-73.9863,B02598,232,Manhattan,Two Bridges/Seward Park
1,2014-04-01 00:00:00,40.7637,-73.9600,B02598,141,Manhattan,Lenox Hill West
2,2014-04-01 00:00:00,40.7215,-73.9952,B02682,144,Manhattan,Little Italy/NoLiTa
3,2014-04-01 00:01:00,40.7355,-73.9966,B02617,113,Manhattan,Greenwich Village North
4,2014-04-01 00:02:00,40.7184,-73.9601,B02682,255,Brooklyn,Williamsburg (North Side)


In [ ]:
uber_all.tail()

,datetime,lat,lon,base,locationID,borough,zone
18715141,2015-06-30 23:59:00,40.7580,-73.9777,B02682,161,Manhattan,Midtown Center
18715142,2015-06-30 23:59:00,40.8043,-73.9513,B02682,41,Manhattan,Central Harlem
18715143,2015-06-30 23:59:00,40.7109,-73.9599,B02682,256,Brooklyn,Williamsburg (South Side)
18715144,2015-06-30 23:59:00,40.7283,-73.9974,B02598,114,Manhattan,Greenwich Village South
18715145,2015-06-30 23:59:00,40.6776,-73.9676,B02003,189,Brooklyn,Prospect Heights


# Split datetime into date and time

### Split datetime into date and time

In [ ]:
# Create date column
uber_all["date"] = uber_all["datetime"].apply(lambda x: x.date())

In [ ]:
# Create time column
uber_all["time"] = uber_all["datetime"].apply(lambda x: x.time())

In [ ]:
uber_all.head()

,datetime,lat,lon,base,locationID,borough,zone,date,time
0,2014-04-01 00:00:00,40.7188,-73.9863,B02598,232,Manhattan,Two Bridges/Seward Park,2014-04-01,00:00:00
1,2014-04-01 00:00:00,40.7637,-73.9600,B02598,141,Manhattan,Lenox Hill West,2014-04-01,00:00:00
2,2014-04-01 00:00:00,40.7215,-73.9952,B02682,144,Manhattan,Little Italy/NoLiTa,2014-04-01,00:00:00
3,2014-04-01 00:01:00,40.7355,-73.9966,B02617,113,Manhattan,Greenwich Village North,2014-04-01,00:01:00
4,2014-04-01 00:02:00,40.7184,-73.9601,B02682,255,Brooklyn,Williamsburg (North Side),2014-04-01,00:02:00


### Save csv with date and time split 

In [ ]:
# uber_all.to_csv(os.path.join(dc_data_directory, "uber_all_processed_date_time_split.csv"), index = False)

### Confirm format 

In [ ]:
uber_all_timesplit = pd.read_csv(os.path.join(dc_data_directory, "uber_all_processed_date_time_split.csv"), parse_dates=["datetime", "date"])

In [ ]:
uber_all_timesplit.head()

,datetime,lat,lon,base,locationID,borough,zone,date,time
0,2014-04-01 00:00:00,40.7188,-73.9863,B02598,232,Manhattan,Two Bridges/Seward Park,2014-04-01,00:00:00
1,2014-04-01 00:00:00,40.7637,-73.9600,B02598,141,Manhattan,Lenox Hill West,2014-04-01,00:00:00
2,2014-04-01 00:00:00,40.7215,-73.9952,B02682,144,Manhattan,Little Italy/NoLiTa,2014-04-01,00:00:00
3,2014-04-01 00:01:00,40.7355,-73.9966,B02617,113,Manhattan,Greenwich Village North,2014-04-01,00:01:00
4,2014-04-01 00:02:00,40.7184,-73.9601,B02682,255,Brooklyn,Williamsburg (North Side),2014-04-01,00:02:00


In [ ]:
uber_all_timesplit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18715146 entries, 0 to 18715145
Data columns (total 9 columns):
 #   Column      Dtype         
---  ------      -----         
 0   datetime    datetime64[ns]
 1   lat         float64       
 2   lon         float64       
 3   base        object        
 4   locationID  int64         
 5   borough     object        
 6   zone        object        
 7   date        datetime64[ns]
 8   time        object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(4)
memory usage: 1.3+ GB
